In [26]:
import pandas as pd
import nsepy as ns
from datetime import date
from nsepy import get_history
import numpy as np
from pathlib import Path
import scipy.integrate as integrate

In [92]:
df = pd.read_csv("/home/harshit/Project/sec bhavdata.csv")  # making list to select companies returns

In [108]:
factors = pd.read_csv("/home/harshit/Project/Returns/averages/FourFactors.csv")
Rf=pd.DataFrame(factors[["Date","Rf %"]])
Rf

,Date,Rf %
0,20130101,0.031492
1,20130102,0.031492
2,20130103,0.031159
3,20130104,0.031159
4,20130107,0.031159
5,20130108,0.031159
6,20130109,0.031159
7,20130110,0.031011
8,20130111,0.031011
9,20130114,0.031011


In [109]:
entropy = pd.DataFrame(columns=["Symbol","Entropy"])
renyi_entropy = pd.DataFrame(columns=["Symbol","Renyi_entropy"])

for i in range(len(df)):
    symbol = df.iloc[i]["Symbol"]
    path= "/home/harshit/Project/risk_free_ret/2017/" # using risk free returns to calculate entropy
    address = Path(path+symbol+".csv")
    if address.is_file():
        equity = pd.read_csv(path+symbol+".csv")
        equity.Date = equity.Date.apply(lambda x: x.replace("-",""))
        equity["Date"] = equity["Date"].apply(pd.to_numeric)
        equity=equity.drop([0])
        equity = pd.merge(Rf,equity,on="Date")
        equity["Ret"] = equity["Return"] - equity["Rf %"]
        
        mean = equity["Ret"].mean()  
        var  = equity["Ret"].var()
        a= equity["Ret"].min() 
        b=equity["Ret"].max()
        def pdf(x,var,mean):  # fxn for continuous shanon entropy
            pf = np.exp(-(x-mean)**2/(2*var))/np.sqrt(2*np.pi*var)
            return pf*(np.log(pf))
        
        ent = -integrate.quad(pdf,a,b,args=(var,mean))[0] #Shanon entropy
        
        def pdf2(x,var,mean):  # fxn for continuous shanon entropy
            pf = np.exp(-(x-mean)**2/(2*var))/np.sqrt(2*np.pi*var)
            return pf**2
        
        ent2=  -np.log(integrate.quad(pdf2,a,b,args=(var,mean))[0]) #Renyi entropy 
        df2 = pd.DataFrame([[symbol,ent]],columns=["Symbol","Entropy"])
        df3 = pd.DataFrame([[symbol,ent2]],columns=["Symbol","Renyi_entropy"])
        entropy = entropy.append(df2,ignore_index=True)
        renyi_entropy = renyi_entropy.append(df3,ignore_index=True)

entropy = entropy.sort_values("Entropy",ascending=0)  # making a sigle csv for all entropies
renyi_entropy = renyi_entropy.sort_values("Renyi_entropy",ascending=0)
entropy.to_csv(path+"continuous_entropy.csv",index=False)
renyi_entropy.to_csv(path+"continuous_renyi_entropy.csv", index=False)

,Symbol,Renyi_entropy
245,DPL,6.298086
558,KOTAKNIFTY,3.407386
62,ARCOTECH,3.253744
196,COMPINFO,3.171095
250,DWARKESH,3.151773
124,BEL,3.150673
214,DAAWAT,3.145011
457,INFIBEAM,3.141780
147,BLS,3.129384
974,SUTLEJTEX,3.120191
